<a href="https://colab.research.google.com/github/selva-mani-007/gen-ai/blob/main/openai_chatbot_and_resume_analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

API_KEY = "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"
API_URL = "https://openrouter.ai/api/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "your-website.com",  # Change this to your site or repo
    "X-Title": "Your Chatbot Name"
}

def chat():
    conversation = []

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Chat ended.")
            break

        conversation.append({"role": "user", "content": user_input})

        data = {
            "model": "gpt-3.5-turbo",  # Change model if needed
            "messages": conversation
        }

        try:
            response = requests.post(API_URL, json=data, headers=headers)
            response.raise_for_status()  # Raise error for bad requests
            bot_reply = response.json()["choices"][0]["message"]["content"]

            print(f"Bot: {bot_reply}\n")
            conversation.append({"role": "assistant", "content": bot_reply})

        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            break

chat()

You: hi
Bot: Hello! How can I help you today?

You: how are you
Bot: I'm just a computer program, so I don't have feelings or emotions. But I'm here to assist you with anything you need. How can I help you today?

You: do you know tamil
Bot: Yes, I am familiar with the Tamil language. I can help you with translations or any other information you may need in Tamil. Just let me know how I can assist you.

You: do know the capital of India
Bot: Yes, the capital of India is New Delhi.

You: exit
Chat ended.


In [ ]:
!pip install spacy pandas PyPDF2 scikit-learn
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 44.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [14]:
import re
import spacy
import PyPDF2
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Load NLP Model
nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(pdf_path):
    """Extract text from PDF file."""
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

def extract_contact_info(text):
    """Extracts email and phone number."""
    email = re.findall(r'\S+@\S+', text)
    phone = re.findall(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', text)
    return email[0] if email else None, phone[0] if phone else None

def extract_skills(text):
    """Extract skills using NLP."""
    skills = []
    doc = nlp(text)
    for token in doc:
        if token.pos_ in ["NOUN", "PROPN"]:
            skills.append(token.text)
    return list(set(skills))

def match_resume_with_job(resume_text, job_description):
    """Matches resume skills with job description."""
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([resume_text, job_description])
    similarity = (tfidf_matrix * tfidf_matrix.T).toarray()[0,1]
    return round(similarity * 100, 2)

def analyze_resume(pdf_path, job_description):
    """Analyzes the resume and returns extracted details."""
    text = extract_text_from_pdf(pdf_path)
    email, phone = extract_contact_info(text)
    skills = extract_skills(text)
    score = match_resume_with_job(text, job_description)

    return {
        "Email": email,
        "Phone": phone,
        "Skills": skills,
        "Match Score": f"{score}%"
    }

# Example Usage
job_description = "Gen"
pdf_path = "/content/VijaiSuriaM_Resume.pdf"  # Change to actual file path

result = analyze_resume(pdf_path, job_description)
print(pd.DataFrame([result]))


                     Email Phone  \
0  @vijaisuria87@gmail.com  None   

                                              Skills Match Score  
0  [modal, booking, EmoTrack, canteen, C++, Prese...       2.39%  
